<a href="https://colab.research.google.com/github/Jum-eng/Social-Media-Mining-Final_Project/blob/main/Multimodal_Sentiment_Anlysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, GlobalAveragePooling2D, Input, Concatenate
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split


In [2]:
data = pd.read_csv('/content/Mental-Health-Twitter.csv')
data

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1013187241,84,211,251,837,0,1
1,1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1013187241,84,211,251,837,1,1
2,2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1013187241,84,211,251,837,0,1
3,3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1013187241,84,211,251,837,2,1
4,4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1013187241,84,211,251,837,1,1
...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,819336825231773698,Thu Jan 12 00:14:56 +0000 2017,A day without sunshine is like night.,1169875706,442,230,7,1063601,0,0
19996,19996,819334654260080640,Thu Jan 12 00:06:18 +0000 2017,"Boren's Laws: (1) When in charge, ponder. (2) ...",1169875706,442,230,7,1063601,0,0
19997,19997,819334503042871297,Thu Jan 12 00:05:42 +0000 2017,The flow chart is a most thoroughly oversold p...,1169875706,442,230,7,1063601,0,0
19998,19998,819334419374899200,Thu Jan 12 00:05:22 +0000 2017,"Ships are safe in harbor, but they were never ...",1169875706,442,230,7,1063601,0,0


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Function to encode text using BERT
def encode_text(text):
  encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
  with torch.no_grad():
    model_output = model(**encoded_input)
  embeddings = model_output.last_hidden_state[:, 0, :].numpy()
  return embeddings

# Apply BERT encoding to 'post_text' column
data['embeddings'] = data['post_text'].apply(lambda x: encode_text(x))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
data = data.drop(['Unnamed: 0'], axis=1)
data

,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1013187241,84,211,251,837,0,1
1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1013187241,84,211,251,837,1,1
2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1013187241,84,211,251,837,0,1
3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1013187241,84,211,251,837,2,1
4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1013187241,84,211,251,837,1,1
...,...,...,...,...,...,...,...,...,...,...
19995,819336825231773698,Thu Jan 12 00:14:56 +0000 2017,A day without sunshine is like night.,1169875706,442,230,7,1063601,0,0
19996,819334654260080640,Thu Jan 12 00:06:18 +0000 2017,"Boren's Laws: (1) When in charge, ponder. (2) ...",1169875706,442,230,7,1063601,0,0
19997,819334503042871297,Thu Jan 12 00:05:42 +0000 2017,The flow chart is a most thoroughly oversold p...,1169875706,442,230,7,1063601,0,0
19998,819334419374899200,Thu Jan 12 00:05:22 +0000 2017,"Ships are safe in harbor, but they were never ...",1169875706,442,230,7,1063601,0,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   post_id       20000 non-null  int64 
 1   post_created  20000 non-null  object
 2   post_text     20000 non-null  object
 3   user_id       20000 non-null  int64 
 4   followers     20000 non-null  int64 
 5   friends       20000 non-null  int64 
 6   favourites    20000 non-null  int64 
 7   statuses      20000 non-null  int64 
 8   retweets      20000 non-null  int64 
 9   label         20000 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 1.5+ MB


In [ ]:
data.isnull().sum()

,0
post_id,0
post_created,0
post_text,0
user_id,0
followers,0
friends,0
favourites,0
statuses,0
retweets,0
label,0


In [ ]:
data.duplicated().sum()

117

In [ ]:
data.duplicated().sum()

117

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.duplicated().sum()

0

In [ ]:
data_2 = pd.read_csv('/content/city_temperature.csv')
data_2

In [ ]:
data_3 = pd.read_csv('/content/fer2013.csv')
data_3

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
33863,6,46 45 37 12 20 25 12 19 50 41 25 22 25 24 32 4...,PrivateTest
33864,4,53 84 100 131 143 145 125 131 134 143 124 104 ...,PrivateTest
33865,6,197 198 192 176 164 81 88 119 169 141 120 141 ...,PrivateTest
33866,5,255 239 148 137 115 120 162 169 116 152 109 11...,PrivateTest


In [ ]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33868 entries, 0 to 33867
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  33868 non-null  int64 
 1   pixels   33868 non-null  object
 2   Usage    33867 non-null  object
dtypes: int64(1), object(2)
memory usage: 793.9+ KB


In [ ]:
data_3.describe()

,emotion
count,33868.000000
mean,3.323964
std,1.872461
min,0.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,6.000000


In [ ]:
data_3.isnull().sum()

,0
emotion,0
pixels,0
Usage,1


In [ ]:
data_3.duplicated().sum()

1220

In [ ]:
data_3.drop_duplicates(inplace=True)

In [ ]:
data_3.duplicated().sum()

0

In [ ]:
combined_data = pd.concat([data, data, data_3], axis=1)
combined_data


,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label,...,user_id,followers,friends,favourites,statuses,retweets,label,emotion,pixels,Usage
0,6.378947e+17,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1.013187e+09,84.0,211.0,251.0,837.0,0.0,1.0,...,1.013187e+09,84.0,211.0,251.0,837.0,0.0,1.0,0.0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,6.378904e+17,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1.013187e+09,84.0,211.0,251.0,837.0,1.0,1.0,...,1.013187e+09,84.0,211.0,251.0,837.0,1.0,1.0,0.0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,6.377493e+17,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1.013187e+09,84.0,211.0,251.0,837.0,0.0,1.0,...,1.013187e+09,84.0,211.0,251.0,837.0,0.0,1.0,2.0,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,6.376964e+17,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1.013187e+09,84.0,211.0,251.0,837.0,2.0,1.0,...,1.013187e+09,84.0,211.0,251.0,837.0,2.0,1.0,4.0,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6.376963e+17,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1.013187e+09,84.0,211.0,251.0,837.0,1.0,1.0,...,1.013187e+09,84.0,211.0,251.0,837.0,1.0,1.0,6.0,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,46 45 37 12 20 25 12 19 50 41 25 22 25 24 32 4...,PrivateTest
33864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,53 84 100 131 143 145 125 131 134 143 124 104 ...,PrivateTest
33865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,197 198 192 176 164 81 88 119 169 141 120 141 ...,PrivateTest
33866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,255 239 148 137 115 120 162 169 116 152 109 11...,PrivateTest


In [ ]:
# prompt: drop rows with NaN

combined_data.dropna(inplace=True)


In [ ]:
combined_data.isnull().sum()

,0
post_id,0
post_created,0
post_text,0
user_id,0
followers,0
friends,0
favourites,0
statuses,0
retweets,0
label,0


In [ ]:
combined_data.dropna(inplace=True)
combined_data


NameError: name 'combined_data' is not defined